# Failure analysis of a MAPDL (RST) model {#rst_workflow_example}

This example shows the postprocessing of an MAPDL (RST) model with
layered elements that was not preprocessed by ACP. The difference
between the RST-only and ACP-based workflow is that the section data are
loaded from the RST file instead of the ACP layup file. This happens
automatically if the parameter [composite]{.title-ref} of the
`.ContinuousFiberCompositesFiles`{.interpreted-text role="class"} class
is not set.

The engineering data file (XML or ENGD) with the material properties is
needed anyway. Otherwise, the material properties cannot be mapped. At
the end of this example, two workflows are shown on how to create the
engineering data file based on a MAPDL model and how to set the material
UUIDs in MAPDL.

::: important
::: title
Important
:::

The material UUIDs in the engineering data file must be identical to the
UUIDs in Mechanical APDL (RST file). A detailed explanation can be found
at the end of this example.
:::

The postprocessing of MAPDL models is supported in 2024 R2 (DPF Server
version 8.0) and later. A few advanced features are not supported with
the RST only workflow. For more information, see
`limitations`{.interpreted-text role="ref"}.

::: note
::: title
Note
:::

When using a Workbench project, use the
`.composite_files_from_workbench_harmonic_analysis`{.interpreted-text
role="func"} method to obtain the input files.
:::


# Set up analysis

Setting up the analysis consists of loading Ansys libraries, connecting
to the DPF server, and retrieving the example files.

Load Ansys libraries.


In [ ]:
import ansys.dpf.core as dpf

from ansys.dpf.composites.composite_model import CompositeModel
from ansys.dpf.composites.constants import FailureOutput
from ansys.dpf.composites.example_helper import get_continuous_fiber_example_files
from ansys.dpf.composites.failure_criteria import (
    CombinedFailureCriterion,
    CoreFailureCriterion,
    FaceSheetWrinklingCriterion,
    MaxStrainCriterion,
    MaxStressCriterion,
    VonMisesCriterion,
)
from ansys.dpf.composites.select_indices import get_selected_indices
from ansys.dpf.composites.server_helpers import connect_to_or_start_server

Start a DPF server and copy the example files into the current working
directory.


In [ ]:
server = connect_to_or_start_server()

Get input files (RST and material.engd but skip the ACP layup file).


In [ ]:
composite_files_on_server = get_continuous_fiber_example_files(server, "shell", True)
print(composite_files_on_server)

# Configure combined failure criterion

Configure the combined failure criterion.


In [ ]:
combined_fc = CombinedFailureCriterion(
    name="failure of all materials",
    failure_criteria=[
        MaxStrainCriterion(),
        MaxStressCriterion(),
        CoreFailureCriterion(),
        VonMisesCriterion(vme=True, vms=False),
        FaceSheetWrinklingCriterion(),
    ],
)

# Set up model and evaluate failures

Set up the composite model.


In [ ]:
composite_model = CompositeModel(composite_files_on_server, server)

Evaluate failures for the entire model


In [ ]:
output_all_elements = composite_model.evaluate_failure_criteria(
    combined_criterion=combined_fc,
)
irf_field = output_all_elements.get_field({"failure_label": FailureOutput.FAILURE_VALUE})
irf_field.plot()

# Create and plot a sampling point


In [ ]:
import matplotlib.pyplot as plt

sampling_point = composite_model.get_sampling_point(combined_criterion=combined_fc, element_id=2)
sampling_plot = sampling_point.get_result_plots(core_scale_factor=0.1)
plt.show()

# Layer-wise failure criteria

The next lines show how to compute layer-wise failure criteria data and
how to access it. Ply-wise filtering (by analysis ply name) is not
supported in the RST workflow because the plies are not available in the
RST file.


First, the inputs for the multiple failure criteria operator have to be
prepared.


In [ ]:
strain_operator = composite_model.core_model.results.elastic_strain()
strain_operator.inputs.bool_rotate_to_global(False)

stress_operator = composite_model.core_model.results.stress()
stress_operator.inputs.bool_rotate_to_global(False)

Then, the operator is initialized and the inputs are connected. Note:
the output of the multiple failure criteria operator has the same format
as the input (strain and stress fields). So there is one failure value
and mode for each integration point, layer and element, for all time
steps.


In [ ]:
failure_evaluator = dpf.Operator("composite::multiple_failure_criteria_operator")
failure_evaluator.inputs.configuration(combined_fc.to_json())
failure_evaluator.inputs.materials_container(
    composite_model.material_operators.material_provider.outputs
)
failure_evaluator.inputs.strains_container(strain_operator.outputs.fields_container)
failure_evaluator.inputs.stresses_container(stress_operator.outputs.fields_container)
failure_evaluator.inputs.mesh(composite_model.get_mesh())
failure_evaluator.inputs.section_data_container(
    composite_model.get_layup_operator().outputs.section_data_container
)
irf_field = failure_evaluator.outputs.fields_container.get_data().get_field(
    {"failure_label": FailureOutput.FAILURE_VALUE, "time": 1}
)

failure_mode_field is not used further, but it is demonstrated how to
access it.


In [ ]:
failure_mode_field = failure_evaluator.outputs.fields_container.get_data().get_field(
    {"failure_label": FailureOutput.FAILURE_MODE, "time": 1}
)

This example computes a \"failure intensity\" which is in this case the
difference between the maxima of the most critical and the lowest
critical layer. Or in other words, it shows how uniform the laminate is
loaded. A high intensity means there is a big difference between the
maxima failure values of the layers.


In [ ]:
failure_intensity_field = dpf.field.Field(
    location=dpf.locations.elemental, nature=dpf.natures.scalar
)

with failure_intensity_field.as_local_field() as local_field:
    for element_id in irf_field.scoping.ids:
        irf_data = irf_field.get_entity_data_by_id(element_id)
        element_info = composite_model.get_element_info(element_id)

        most_critical_layer_irf = 0.0
        lowest_critical_layer_irf = 1e9

        for layer_index, dpf_material_id in enumerate(element_info.dpf_material_ids):
            selected_indices = get_selected_indices(element_info, layers=[layer_index])
            layer_irf_values = irf_data[selected_indices]
            max_value = layer_irf_values.max()
            if max_value > most_critical_layer_irf:
                most_critical_layer_irf = max_value
            if max_value < lowest_critical_layer_irf:
                lowest_critical_layer_irf = max_value

        local_field.append([most_critical_layer_irf - lowest_critical_layer_irf], element_id)

composite_model.get_mesh().plot(failure_intensity_field)

# Create Engineering Data file and set material UUIDs in MAPDL

Here are two workflows for setting up the engineering data file and the
material UUIDs in MAPDL. The material UUIDs must be set in MAPDL before
the model is solved.

With WB and Engineering Data:

:   -   Create an External Model system in WB and load the solver input
        file
    -   Link the External Model with an Engineering Data system and
        update it
    -   Save the project and copy the generated engineering data file
        (EngineeringData.xml)
    -   For each material, look for the `DataTransferID`, go to MAPDL
        and set the material UUIDs with the
        `MP,UVID,<material index>,<value>` command

With ACP Standalone (for constant material properties only):

:   -   Start ACP, go to *File - Import Model* and load the solver input
        file (CDB)
    -   Go to the Materials folder and export the engineering data file
        (Ansys Workbench XML)
    -   For each material, look for the `DataTransferID`, go to MAPDL
        and set the material UUID with the
        `MP,UVID,<material index>,<value>` command.
